# **Modelo de machine learning para determinación de comportamiento en clientes de la empresa MEGALINE**

<p> La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.</p>

<p> Se tiene acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto de Análisis estadístico de datos). </p>

<p>
Para esta tarea de clasificación se busca crear un modelo que escoja el plan correcto para cada usuario. 
</p>

## **Descripción de los datos** 


<p>Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:</p>

* `сalls` — número de llamadas,
* `minutes` — duración total de la llamada en minutos,
* `messages` — número de mensajes de texto,
* `mb_used` — Tráfico de Internet utilizado en MB,
* `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).

## **Librerías** 




In [16]:
import pandas as pd                                         #Librería para leer el DataSet

from sklearn.tree import DecisionTreeClassifier             #Modelo arbol de clasificación
from sklearn.ensemble import RandomForestClassifier         #Modelo arbol de clasificación aleatorio
from sklearn.linear_model import LogisticRegression         #Modelo de regreción logística

from sklearn.model_selection import train_test_split        #Para dividir en conjunto de prueba y entrenamiento
from sklearn.metrics import accuracy_score                  #Para medir la exactitud del modelo
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix


## **Visualización de la data** 

<p>Primeramente se procede a observar la data, para luego proceder a la determinación del modelo ótimo de clasificación necesario</p>

In [17]:
users = pd.read_csv('/content/drive/MyDrive/practicum/Sprint7/users_behavior.csv')
users.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Se procede a verificar los valores ausentes y tipos de dato

In [18]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Todo se encuentra en orden por lo que se procede a relaizar lo modelos pertinentes

## **Modelo de Machine Learning** 

### **Segmentación del conjunto de datos**

<p>Lo que se busca en este paso, es desarrollar 3 tipos diferentes de modelos de predicción para la clasifiación y luego determinar cual es el más exacto y preciso con una sensibilidad razonable</p> 

<p>Primeramente Observamos la cantidad de caracteristicas (columnas) y de observaciones (filas) que tiene nuestro DataSet.</p>

In [19]:
users.shape

(3214, 5)

<p>Como se puede observar, se cuenta con 3214 observaciones y 5 características</p>
<p>Ahora, se procede a, separar nmuestro modelo en 3 conjuntos:</p>

* Entrenamiento: `users_train` (60% de los datos)
* Validación: `users_valid` (20% de los datos) 
* Prueba: `users_probe` (20% de los datos)

In [20]:
features = users.drop('is_ultra', axis = 1)
target = users['is_ultra']

features_train, features_valid,  target_train, target_valid=  train_test_split(features, target, test_size = 0.40, random_state=12345)
features_valid, features_probe,  target_valid, target_probe=  train_test_split(features_valid, target_valid, test_size = 0.50, random_state=12345)


Se observa el tamaño de cada conjunto de datos:

In [21]:
features_train.shape

(1928, 4)

In [22]:
features_valid.shape

(643, 4)

In [23]:
features_probe.shape

(643, 4)

Se paramos cada conjunto en sus caracteristicas (features) que el modelo utilizará para predecir los objetivos (target) que en este caso es el tipo de plan al que debe pertenecer cada persona

Ahora, se procede a realizar los 3 modelos para deteminar cual de ellos arroja los mejores resultados:

* Árbol de decisión
* Bosque aleatorio de decisión
* Regresón logística

### **Árbol de decisiones**

Se procede a realizar un modelo de arbol de decisión, para el cual se decide variar el hiperparámetro de la profundidad del mismo con un ciclo `for`, para obtener el resultado más óptimo.

In [24]:
best_model_ad = None
best_result_ad = 0
best_depth_ad = 0
best_presicion = 0
best_recall = 0
for depth in range(1, 11):
  model_ad = DecisionTreeClassifier(random_state=12345, max_depth= depth)
  model_ad.fit(features_train,target_train)
  result = model_ad.score(features_valid,target_valid)
  precision = precision_score(model_ad.predict(features_valid),target_valid)
  recall = recall_score(model_ad.predict(features_valid),target_valid)

  if result > best_result_ad:
    best_model_ad = model_ad
    best_depth_ad = depth
    best_result_ad = result
    best_presicion =  precision
    best_recall = recall  
    
  
print(f'El mejor modelo modelo cuenta con {round(best_result_ad*100,4)}% de exactitud\n\nEmpleando:\nmax_depth = {best_depth_ad} \n')
print(f'La precisión del modelo es: {round(best_presicion*100,4)}%\n')
print(f'La sensibilidad del modelo es: {round(best_recall*100,4)}%')

El mejor modelo modelo cuenta con 78.5381% de exactitud

Empleando:
max_depth = 3 

La precisión del modelo es: 43.9153%

La sensibilidad del modelo es: 72.1739%


#### **Conlusión intermedia** 



<p>Como se observa, el mejor modelo se consiguió con un árbol con 3 niveles de profundidad. El mismo tiene una muy buena exactitud de 78.5381%, con una precisión 43.9153%, con una sensibilidad del 72.1739%.</p>

<p>El modelo se podría decir que no está sobreajustado, y logra encontrar al 72% de los valores que efectivamente pertenecen al plan ultra, por esta misma razón la presición es de un 44% ya que se está toamando un conjunto de datos grande, y de ellos solo una fracción son verdaderamente son de este plan. </p>

<p>En este caso está bien, ya que nos interesa tomar en cuenta una gran población de data y detemrinar quiene spueden o no ser del plan ultra y a costa de ellos se comenten algunos errores </p>

### **Bosque aleatorio de decisiones**  



In [25]:
best_model_bd = None
best_n_estimators = 0
best_result_bd = 0
best_depth_bd = 0
best_presicion = 0
best_recall = 0
for estim in range(1,52,10):
  for depth in range(1, 11):
    model_bd = RandomForestClassifier(random_state=12345, n_estimators = estim, max_depth = depth) 
    model_bd.fit(features_train,target_train)
    result = model_bd.score(features_valid, target_valid)
    precision = precision_score(model_bd.predict(features_valid),target_valid)
    recall = recall_score(model_bd.predict(features_valid),target_valid)

    if result > best_result_bd:
      best_model_bd = model_bd
      best_depth_bd = depth
      best_n_estimators = estim
      best_result_bd = result   
      best_presicion =  precision
      best_recall = recall  
print(f'El mejor modelo cuenta con un {round(best_result_bd*100,4)}% de exactitud\n\nEmpleando:\nmax_depth = {best_depth_bd}\nn_estimators = {best_n_estimators}\n ')
print(f'La precisión del modelo es: {round(best_presicion*100,4)}%\n')
print(f'La sensibilidad del modelo es: {round(best_recall*100,4)}%\n')

El mejor modelo cuenta con un 80.7154% de exactitud

Empleando:
max_depth = 8
n_estimators = 41
 
La precisión del modelo es: 48.6772%

La sensibilidad del modelo es: 77.3109%



#### **Conlusión intermedia**

<p>Como se observa, el mejor modelo se consiguió con un bosque de árboles que cuenta con 8 niveles de profundidad y 41 estimadores. Con este modelo se logró obtener una exactitud de 80.7154%, con una precisión 48.6772%, y una sensibilidad del 77.3109%.</p>

<p>El modelo no está sobreajustado, y logra encontrar al 77% de los valores que efectivamente pertenecen al plan ultra, mucho mejor que en el modelo del arbol decisión, por esta misma razón la precisión es de un 48% sin embargo es más alta que en el caso anterior. </p>

<p>Este modelo es el seleccionado por ajora, ya que cumple con los objetivos que nos planteamos, aunque es un tanto lento, la cantidad de data no estan elevada como para que esto sea un factor determinante. </p>

### **Regresión logística** 

In [26]:
model_rl = LogisticRegression(random_state=12345, solver='liblinear')
model_rl.fit(features_train,target_train) 
score_valid = model_rl.score(features_valid,target_valid) 
precision = precision_score(model_rl.predict(features_valid),target_valid)
recall = recall_score(model_rl.predict(features_valid),target_valid)

print(f'El modelo tiene una exactitud de: {round(score_valid*100,4)}%\n')
print(f'La presicion del modelo es: {round(precision*100,4)}%\n')
print(f'La sensibilidad del modelo es: {round(recall*100,4)}%\n')

El modelo tiene una exactitud de: 75.8942%

La presicion del modelo es: 22.7513%

La sensibilidad del modelo es: 82.6923%



#### **Conlusión intermedia** 

<p>Como se observa, en este caso se obtuvo la exactitud más baja, con un 75.8942%, una presición que deja bastante que desear de un 22.7513% y una sensibilidad elevada de un 82.6923% con lo cual se puede decir que no cumple como candidato para el caso de estudio</p>


### **Prueba al mejor modelo** 

Ahora volvemos a probar el modelo, pero esta vez con el conjunto de prueba, para determinar si los valores sigue sinedo aceptables

In [27]:
result = best_model_bd.score(features_probe, target_probe)
precision = precision_score(best_model_bd.predict(features_probe),target_probe)
recall = recall_score(best_model_bd.predict(features_probe),target_probe)

print(f'El modelo tiene una exactitud de: {round(result *100,4)}%\n')
print(f'La precisión del modelo es: {round(best_presicion*100,4)}%\n')
print(f'La sensibilidad del modelo es: {round(best_recall*100,4)}%\n')

El modelo tiene una exactitud de: 79.7823%

La precisión del modelo es: 48.6772%

La sensibilidad del modelo es: 77.3109%



EN este caso disminuyó un poco la el resultado, sin embargo, sigue siendo un mejor candidato que los anteriores. Por lo que ahora se procede a entrenar el modelo con todos los datos, para mejorarlo y volver a probar. Para ello, vamos a unir el conjunto de `features_train` con `features_valid` y del mismo modo a `target_train` con `target_valid` para luego volver a ahcer la prueba con: `features_probe` y `target_probe`

In [34]:
feaures_train_2 = pd.concat([features_train,features_valid], axis = 0)
target_train_2 = pd.concat([target_train,target_valid], axis = 0)

Se procede a entrenar y probar el modelo

In [35]:
model_bd = RandomForestClassifier(random_state=12345, n_estimators = 41, max_depth = 8) 
model_bd.fit(feaures_train_2,target_train_2)
result = model_bd.score(features_probe, target_probe)

precision = precision_score(model_bd.predict(features_probe),target_probe)
recall = recall_score(model_bd.predict(features_probe),target_probe)
matrix = confusion_matrix(model_bd.predict(features_probe),target_probe)

print(f'El modelo tiene una exactitud de: {round(result *100,4)}%\n')
print(f'La presicion del modelo es: {round(precision*100,4)}%\n')
print(f'La sensibilidad del modelo es: {round(recall*100,4)}%\n')
print(f'La matriz de confusión del modelo es:\n\n{matrix}')

El modelo tiene una exactitud de: 79.9378%

La presicion del modelo es: 52.2167%

La sensibilidad del modelo es: 76.8116%

La matriz de confusión del modelo es:

[[408  97]
 [ 32 106]]


#### **Conlusión intermedia**

Al aumentar la cantidad de datos para el entrenamiento, los resultados mejoraron substancialmente, llegando a un 79.93% de exactitud, 52.22% de precisión y una sensibilidad 76.81%, siendo un modelo un poco mejor que el anterior, lo cual es excelente, así mismo se muestra matriz de confusión de la cual salen los valores antes mencionados, pudienco concluir que el modelo puede ser utilizado para realizar predicciones, cercanas a la realidad.

## **Conclusión General**

<p> 
La compañía móvil Megaline requería desarrollar un modelo capaz de analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
</p>

<p> 
Para ello se realizaron pruebas con 3 modelos diferentes de clasificación, para poder determinar cual es ellos arrojó mejores resultados. Y así detemrinar basado en la data suministrada, cuales son los planes óptimos para cada usuario.</p>

<p>
El arbol de decisión en su mejor modelo consiguió una muy buena exactitud de 78.5381%, con una precisión 43.9153%, y una sensibilidad del 72.1739%, lograno un buen modelo, más no fué el elegido.
</p>

<p>
Así mismo, empleando un bosque de árboles de decisón alreatorio que cuenta con 8 niveles de profundidad y 41 estimadores se logró obtener una exactitud de 80.7154%, con una precisión 48.6772%, y una sensibilidad del 77.3109%. Los cuales fueron valores excelentes para el objetivo que se quiere. Aunque un poco más lento nque la opción anterior, es razonable.
</p>

<p>
Por último la regrsión logística dejo mucho que desear en este caso, con un 75.8942%, una presición de un 22.7513% y una sensibilidad elevada de un 82.6923% con lo cual se puede decir que no cumple como candidato para el caso de estudio
</p>


<p>
Es por ello que se optó por reentrenar el modelo empleando un bosque de árboles de decisón alreatorio el cual como se evidencia en la matriz de confusión tiene valores bastante buenos y nos permite recomendar correctamente a el plan a los clientes.
</p>